In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.framework import graph_util
from os.path import isfile

In [2]:
data_directory = 'data/'
image_directory = 'data/Pre_train/'

In [3]:
ohe = OneHotEncoder()

In [4]:
ohe.fit(np.arange(3).reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [5]:
ohe.transform([[0], [1], [2]]).toarray()

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [6]:
size = 32
batch_size = 256
num_channels = 3
training_iters = 500
display = 10

In [7]:
def read_label_files(csv_file):
    file = open(data_directory+csv_file, 'r')
    filepaths = []
    labels = []
    for i in file:
        filename, label = i.split(',')
        filepaths.append(image_directory+filename)
        labels.append(int(label))
    return filepaths, labels    

In [8]:
train_path, train_labels = read_label_files('train_set.csv')
vali_path, vali_labels = read_label_files('vali_set.csv')

In [9]:
vali_batch_size = len(vali_labels)

In [10]:
train_path_tensor = ops.convert_to_tensor(train_path, dtype=dtypes.string)
vali_path_tensor = ops.convert_to_tensor(vali_path, dtype=dtypes.string)
train_labels_tensor = ops.convert_to_tensor(train_labels, dtype=dtypes.int32)
vali_labels_tensor = ops.convert_to_tensor(vali_labels, dtype=dtypes.int32)

In [11]:
train_input_queue = tf.train.slice_input_producer(
                                    [train_path_tensor, train_labels_tensor],
                                    shuffle=True)
vali_input_queue = tf.train.slice_input_producer(
                                    [vali_path_tensor, vali_labels_tensor],
                                    shuffle=True)

In [12]:
file_content = tf.read_file(train_input_queue[0])
train_image = tf.image.decode_jpeg(file_content, channels=num_channels)
train_image /= 255
train_label = train_input_queue[1]

file_content = tf.read_file(vali_input_queue[0])
vali_image = tf.image.decode_jpeg(file_content, channels=num_channels)
vali_image /= 255
vali_label = vali_input_queue[1]

In [13]:
train_image.set_shape([size, size, num_channels])
vali_image.set_shape([size, size, num_channels])

train_batch = tf.train.batch(
                            [train_image, train_label],
                            batch_size= batch_size
                            )
vali_batch = tf.train.batch(
                            [vali_image, vali_label],
                            batch_size = vali_batch_size
                            )

In [14]:
def freeze_graph(model_folder):

    checkpoint = tf.train.get_checkpoint_state(model_folder)
    input_checkpoint = checkpoint.model_checkpoint_path
    model_name = 'basic_model'
    absolute_model_folder = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_folder + "/" + model_name + ".pb"

    output_node_names = "final_output"

    clear_devices = True

    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()

    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)

        output_graph_def = graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            output_node_names.split(",")
        )

        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))


In [15]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
    
#     for i in range(1):
#         test = sess.run(train_batch)
#     coord.request_stop()
#     coord.join(threads)

In [16]:
# image, label = test

In [17]:
# print(image.shape)
# print(label.shape)

In [18]:
X = tf.placeholder(tf.float32, shape = [None, size, size, 3])
y = tf.placeholder(tf.float32, shape=[None, 3])

In [19]:
weights = {
        'wc1':tf.Variable(tf.random_normal([5, 5, 3, 32])),
        'wc2':tf.Variable(tf.random_normal([5, 5, 32, 64])),
        'wc3':tf.Variable(tf.random_normal([3, 3, 64, 64])),
        'wf1':tf.Variable(tf.random_normal([8 * 8 * 64, 1024])),
        'out':tf.Variable(tf.random_normal([1024, 3]))
         }

In [20]:
bias = {
        'bc1' : tf.Variable(tf.random_normal([32])),
        'bc2' : tf.Variable(tf.random_normal([64])),
        'bc3' : tf.Variable(tf.random_normal([64])),
        'bf1' : tf.Variable(tf.random_normal([1024])),
        'out':tf.Variable(tf.random_normal([3]))
        }

In [21]:
def conv_net(input_image):
        conv1 = tf.nn.conv2d(input_image, weights['wc1'], [1, 1, 1, 1], padding='SAME')
        conv1 = tf.add(conv1, bias['bc1'])
        max1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv2 = tf.nn.conv2d(max1, weights['wc2'], [1, 1, 1, 1], padding='SAME')
        conv2 = tf.add(conv2, bias['bc2'])
        conv3 = tf.nn.conv2d(conv2, weights['wc3'], [1, 1, 1, 1], padding='SAME')
        conv3 = tf.add(conv3, bias['bc3'])
        max2 = tf.nn.max_pool(conv3, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        b, h, w, c = max2.get_shape().as_list()
        unrolled = tf.reshape(max2, [-1, h * w * c])
        full1 = tf.add(tf.matmul(unrolled, weights['wf1']), bias['bf1'])
        out = tf.add(tf.matmul(full1, weights['out']), bias['out'])
        return out

In [22]:
conv_net(tf.random_normal([6, 32, 32, 3]))

<tf.Tensor 'Add_4:0' shape=(6, 3) dtype=float32>

In [23]:
pred = conv_net(X)
output = tf.argmax(pred, axis = 1, name = 'final_output')
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
train_op = tf.train.AdamOptimizer().minimize(loss)
corrected_pred = tf.equal(tf.arg_max(pred, 1), tf.arg_max(y, 1))
acc = tf.reduce_mean(tf.cast(corrected_pred, tf.float32))

In [24]:
saver = tf.train.Saver()

In [27]:
loss_list = []

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    if isfile('models/model.ckpt.index'):
        saver.restore(sess, 'models/model.ckpt')
    val_img, val_lab = sess.run(vali_batch)
    val_lab = ohe.transform(val_lab.reshape(-1, 1)).toarray()
    print(val_lab.shape)
    
    for i in range(training_iters):
        image, label = sess.run(train_batch)
        label = ohe.transform(label.reshape(-1, 1)).toarray()
        temp, _ = sess.run([loss, train_op], feed_dict={X:image, y:label})
        if i % display == 0:
            val_loss, accuracy = sess.run([loss, acc], feed_dict={X:val_img, y:val_lab})
            save_path = saver.save(sess, "models/model.ckpt")
            print('Iter %s, Validation Acc %.4f, Training loss %.4f'%(i, accuracy, val_loss))
    print('Saved and Optimized')
    coord.request_stop()
    coord.join(threads)    

(200, 3)
Iter 0, Validation Acc 0.6000, Training loss 124666.6016
Iter 10, Validation Acc 0.6300, Training loss 119675.0078
Iter 20, Validation Acc 0.6600, Training loss 125022.3828
Iter 30, Validation Acc 0.6700, Training loss 120318.7188
Iter 40, Validation Acc 0.5200, Training loss 177471.4375
Iter 50, Validation Acc 0.5800, Training loss 139528.0000
Iter 60, Validation Acc 0.5150, Training loss 192441.0469
Iter 70, Validation Acc 0.4900, Training loss 158532.8750
Iter 80, Validation Acc 0.6150, Training loss 143947.7031
Iter 90, Validation Acc 0.6050, Training loss 146866.7969
Iter 100, Validation Acc 0.5800, Training loss 135310.7656
Iter 110, Validation Acc 0.6500, Training loss 115331.1016
Iter 120, Validation Acc 0.6150, Training loss 113031.8203
Iter 130, Validation Acc 0.6650, Training loss 129239.4766
Iter 140, Validation Acc 0.5750, Training loss 135996.5938
Iter 150, Validation Acc 0.6150, Training loss 127863.8594
Iter 160, Validation Acc 0.4950, Training loss 173641.1562

In [28]:
freeze_graph('models/')

INFO:tensorflow:Froze 10 variables.
Converted 10 variables to const ops.
37 ops in the final graph.
